# DS 1005 Hwk 3 - Belief Propagation

## Q1 - Sum-product algorithm 

In [1]:
import numpy as np 
import networkx as nx
from fglib import graphs, nodes, rv, inference, utils
from itertools import product
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [2]:
dist_f3 = [0.5, 0.5]
dist_f4 = [0.4,0.6]
px3x4=np.outer(dist_f3,dist_f4)
print(px3x4)
print(px3x4.shape)
px3x4=np.reshape(px3x4, np.shape(px3x4)+(1,))
print(px3x4)
print(px3x4.shape)
px2_conditioned_x3x4=[[[0.2,0.8],
                     [0.25,0.75],],
                     [[0.7,0.3],
                     [0.3,0.7]]]
print(px2_conditioned_x3x4)
dist_f234 =px3x4*px2_conditioned_x3x4
print(dist_f234)
print(dist_f234.shape)
px2= np.sum(dist_f234, axis=(0,1))
print(px2)
px1_conditioned_x2 = [[0.5,0.5],
                     [0.7,0.3]]
dist_f12 =px2[:,np.newaxis]*px1_conditioned_x2
print(dist_f12)
print(dist_f12.shape)

[[0.2 0.3]
 [0.2 0.3]]
(2, 2)
[[[0.2]
  [0.3]]

 [[0.2]
  [0.3]]]
(2, 2, 1)
[[[0.2, 0.8], [0.25, 0.75]], [[0.7, 0.3], [0.3, 0.7]]]
[[[0.04  0.16 ]
  [0.075 0.225]]

 [[0.14  0.06 ]
  [0.09  0.21 ]]]
(2, 2, 2)
[0.345 0.655]
[[0.1725 0.1725]
 [0.4585 0.1965]]
(2, 2)


In [3]:
def make_debug_graph():

    # Create factor graph
    fg = graphs.FactorGraph()

    # Create variable nodes
    x1 = nodes.VNode("x1", rv.Discrete)
    x2 = nodes.VNode("x2", rv.Discrete)
    x3 = nodes.VNode("x3", rv.Discrete)
    x4 = nodes.VNode("x4", rv.Discrete)

    # Create factor nodes
    f12 = nodes.FNode("f12")
    f234 = nodes.FNode("f234")
    f3 = nodes.FNode("f3")
    f4 = nodes.FNode("f4")

    # Add nodes to factor graph
    fg.set_nodes([x1, x2, x3, x4])
    fg.set_nodes([f12, f234, f3,f4 ])

    # Add edges to factor graph
    fg.set_edge(x1, f12)
    fg.set_edge(f12, x2)
    fg.set_edge(x2, f234)
    fg.set_edge(f234, x3)
    fg.set_edge(f234, x4)
    fg.set_edge(x3, f3)
    fg.set_edge(x4, f4)

    #add potential for f_3: p(x3)
    dist_f3 = [0.5, 0.5]
    f3.factor = rv.Discrete(dist_f3,x3)
    
    #add potential for f_4: p(x4)
    dist_f4 = [0.4,0.6]
    f4.factor = rv.Discrete(dist_f4,x4)
    
    # add potential for f_{234}: p(x2, x3, x4) = p(x2|x3,x4) p(x3,x4)
    px3x4=np.outer(dist_f3,dist_f4)
    px3x4=np.reshape(px3x4, np.shape(px3x4)+(1,))
    px2_conditioned_x3x4=[[[0.2,0.8],
                         [0.25,0.75],],
                         [[0.7,0.3],
                         [0.3,0.7]]]
    
    dist_f234 =px3x4*px2_conditioned_x3x4
    f234.factor = rv.Discrete(dist_f234,x3,x4,x2)
   
    # add potential for f_{12}:  p (x1,x2) = p(x1 | x2) p(x2)
    px1_conditioned_x2 = [[0.5,0.5],
                         [0.7,0.3]]
    px2= np.sum(dist_f234, axis=(0,1))
    dist_f12 =px2[:,np.newaxis]*px1_conditioned_x2
    f12.factor = rv.Discrete(dist_f12,x2,x1)
    # Perform sum-product algorithm on factor graph
    # and request belief of variable node x1
    
    belief = inference.sum_product(fg, x1)
    print(belief)
    return (fg)


### 1a. Implement sum-product algorithm 

In [4]:
# # numerically unstable 

# def get_beliefs(model, n_iter=10):
    
#     # initialize variable to factor messages 
#     for v in model.get_vnodes():
#         for f in v.neighbors():
#             initial_msg = rv.Discrete(np.ones(2,), v)
#             model[v][f]['object'].set_message(v, f, initial_msg)

#     # run parallel updates for n_iter times 
#     for i in range(n_iter):
        
#         # update factor-to-variable messages
#         for f in model.get_fnodes():
#             for v in f.neighbors():
#                 msg = f.factor 
#                 for n in f.neighbors(exclusion=v):
#                     msg *= model[n][f]['object'].get_message(n, f)
#                 for n in f.neighbors(exclusion=v):
#                     msg = msg.marginalize(n, normalize=False)
#                 model[f][v]['object'].set_message(f, v, msg)
                
#         # update variable-to-factor messages 
#         for v in model.get_vnodes(): 
#             for f in v.neighbors():
#                 msg = rv.Discrete(np.ones(2,), v)
#                 for n in v.neighbors(exclusion=f):
#                     msg *= model[n][v]['object'].get_message(n, v)
#                 model[v][f]['object'].set_message(v, f, msg)

#         # store beliefs of variable nodes 
#         beliefs = {}
#         for v in model.get_vnodes():
#             beliefs[v] = v.belief(model)
    
#     return beliefs

In [16]:
# def get_beliefs(model, n_iter=10):
    
#     # initialize variable to factor messages 
#     for v in model.get_vnodes():
#         for f in v.neighbors():
#             initial_msg = rv.Discrete(np.ones(2,), v)
#             model[v][f]['object'].set_message(v, f, initial_msg.normalize())

#     # run parallel updates for n_iter times 
#     for i in range(n_iter):
        
#         # update factor-to-variable messages
#         for f in model.get_fnodes():
#             for v in f.neighbors():
#                 msg = f.factor 
#                 for n in f.neighbors(exclusion=v):
#                     msg *= model[n][f]['object'].get_message(n, f)
#                     msg = msg.normalize()
#                 for n in f.neighbors(exclusion=v):
#                     msg = msg.marginalize(n, normalize=True)
#                 model[f][v]['object'].set_message(f, v, msg.normalize())
                
#         # update variable-to-factor messages 
#         for v in model.get_vnodes(): 
#             for f in v.neighbors():
#                 msg = rv.Discrete(np.ones(2,), v)
#                 for n in v.neighbors(exclusion=f):
#                     msg *= model[n][v]['object'].get_message(n, v)
#                     msg = msg.normalize()
#                 model[v][f]['object'].set_message(v, f, msg.normalize())

#         # store beliefs of variable nodes 
#         beliefs = {}
#         for v in model.get_vnodes():
#             belief = rv.Discrete(np.ones(2,), v)
#             for n in v.neighbors():
#                 belief *= model[n][v]['object'].get_message(n, v)
#             beliefs[v] = belief.normalize()
    
#     return beliefs

def get_beliefs_at_iter(beliefs_history, n_iter):
    beliefs_at_iter = {}
    for v in beliefs_history: 
        beliefs_at_iter[v] = beliefs_history[v][n_iter]
    return beliefs_at_iter 

def get_beliefs(model, n_iter=10):
    
    beliefs_history = {v:[] for v in model.get_vnodes()}
    
    # initialize variable to factor messages 
    for v in model.get_vnodes():
        for f in v.neighbors():
            initial_msg = rv.Discrete(np.ones(2,), v)
            model[v][f]['object'].set_message(v, f, initial_msg.normalize())

    # run parallel updates for n_iter times 
    for i in range(n_iter):
        
        # update factor-to-variable messages
        for f in model.get_fnodes():
            for v in f.neighbors():
                msg = f.factor 
                for n in f.neighbors(exclusion=v):
                    msg *= model[n][f]['object'].get_message(n, f)
                    msg = msg.normalize()
                for n in f.neighbors(exclusion=v):
                    msg = msg.marginalize(n, normalize=True)
                model[f][v]['object'].set_message(f, v, msg.normalize())
                
        # update variable-to-factor messages 
        for v in model.get_vnodes(): 
            for f in v.neighbors():
                msg = rv.Discrete(np.ones(2,), v)
                for n in v.neighbors(exclusion=f):
                    msg *= model[n][v]['object'].get_message(n, v)
                    msg = msg.normalize()
                model[v][f]['object'].set_message(v, f, msg.normalize())

        # store beliefs of variable nodes 
        for v in model.get_vnodes():
            belief = rv.Discrete(np.ones(2,), v)
            for n in v.neighbors():
                belief *= model[n][v]['object'].get_message(n, v)
            beliefs_history[v].append(belief.normalize())
            
    beliefs = get_beliefs_at_iter(beliefs_history, n_iter-1)
    
    return beliefs, beliefs_history

In [17]:
# # attempt to take log, but answers don't seem correct

# def get_beliefs_log(model, n_iter=10):
    
#     # initialize variable to factor messages 
#     for v in model.get_vnodes():
#         for f in v.neighbors():
#             initial_msg = rv.Discrete(np.ones(2,), v).normalize().log()
#             model[v][f]['object'].set_message(v, f, initial_msg, logarithmic=True)

#     # run parallel updates for n_iter times 
#     for i in range(n_iter):
        
#         # update factor-to-variable messages
#         for f in model.get_fnodes():
#             for v in f.neighbors():
#                 sum_lambdas = rv.Discrete(np.zeros(2,), v)
#                 for n in f.neighbors(exclusion=v):
#                     sum_lambdas += model[n][f]['object'].get_message(n, f)
#                 max_lambda = max(np.nan_to_num(sum_lambdas.pmf.flatten()))
#                 sum_lambdas = rv.Discrete(sum_lambdas.pmf - max_lambda, v)
#                 msg = rv.Discrete(np.exp(sum_lambdas.pmf), *sum_lambdas.dim)
#                 msg = f.factor * msg
#                 for n in f.neighbors(exclusion=v):
#                     msg = msg.marginalize(n, normalize=False)
#                 msg = msg.log()
#                 msg = rv.Discrete(msg.pmf + max_lambda, v)
#                 msg = msg.normalize()
#                 model[f][v]['object'].set_message(f, v, msg, logarithmic=True)
                
#         # update variable-to-factor messages 
#         for v in model.get_vnodes(): 
#             for f in v.neighbors():
#                 msg = rv.Discrete(np.zeros(2,), v)
#                 for n in v.neighbors(exclusion=f):
#                     msg += model[n][v]['object'].get_message(n, v)
#                 model[v][f]['object'].set_message(v, f, msg, logarithmic=True)

#         # store beliefs of variable nodes 
#         beliefs = {}
#         for v in model.get_vnodes():
#             log_b = v.belief(normalize=False)
#             b = rv.Discrete(np.exp(log_b.pmf), log_b.dim).normalize()
#             beliefs[v] = b
    
#     return beliefs

### 1b. Test on 4-node factor graph provided 

In [20]:
fg = make_debug_graph()
beliefs, _ = get_beliefs(fg)
for (v,b) in beliefs.items():
    print(v, b)

[0.65897284 0.34102716]
x1 [0.65897284 0.34102716]
x2 [0.20513578 0.79486422]
x3 [0.52640912 0.47359088]
x4 [0.28679718 0.71320282]


## Q2 - Low-Density Priority Check (LDPC)

In [21]:
from pyldpc import RegularH, CodingMatrixG # old version of API

### Part a:  

In [22]:
def get_factor_potentials(d_v): # where d_v = num of variable nodes in parity check equation 
    permutations = product([0,1], repeat=d_v)
    phi = np.zeros([2 for i in range(d_v)])
    for idx in permutations:
        if sum(idx) % 2 == 0: 
            phi[idx] = 1
        else:
            phi[idx] = 0
    return phi 

In [23]:
def make_ldpc_fg(H):
    """ Given an arbitrary parity check matrix create a corresponding factor graph """
    
    # initialize factor graph  
    fg = graphs.FactorGraph()
    
    # infer the number of factor and variable nodes required
    num_factors, num_variables = H.shape

    # create factor nodes 
    fnodes = [] 
    for i in range(num_factors):
        fnodes.append(nodes.FNode("f" + str(i)))
    
    # create variable nodes 
    vnodes = [] 
    for j in range(num_variables):
        vnodes.append(nodes.VNode("x" + str(j), rv.Discrete))
        
    # add nodes to factor graph 
    fg.set_nodes(vnodes)
    fg.set_nodes(fnodes)
    
    # add edges to factor graph based on H 
    for i in range(num_factors):
        for j in range(num_variables):
            if H[i,j] == 1:
                fg.set_edge(fnodes[i], vnodes[j])
                
    # set factor potentials 
    for fnode in fg.get_fnodes():
        vnodes2check = [vnode for vnode in fnode.neighbors()]
        fnode_potentials = get_factor_potentials(len(vnodes2check))
        fnode.factor = rv.Discrete(fnode_potentials, *vnodes2check)
        
    return fg

In [24]:
# create parity check matrix (note d_v = ones per column, d_c = ones per row)
H = RegularH(n=8, d_v=2, d_c=4)
print(H)
ldpc_fg = make_ldpc_fg(H)

[[1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]
 [1 0 0 1 1 1 0 0]
 [0 1 1 0 0 0 1 1]]


In [25]:
def evaluate_potential(model, codeword):
    p = 1
    for f in model.get_fnodes():
        v_checked = [int(str(v)[1:]) for v in f.neighbors()]
        idx = tuple([codeword[i] for i in v_checked])
        p *= f.factor.pmf[idx]
    return p

In [26]:
# define invalid codewords 
invalid_code = np.array([1, 0, 0, 0, 0, 0, 0, 0]) # should fail parity check 
print("Given H = {}, this code is expected to fail parity check: {}".format(H, invalid_code))
prob = evaluate_potential(ldpc_fg, invalid_code)
print("Prob is {}".format(prob))

Given H = [[1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]
 [1 0 0 1 1 1 0 0]
 [0 1 1 0 0 0 1 1]], this code is expected to fail parity check: [1 0 0 0 0 0 0 0]
Prob is 0.0


In [27]:
# zero should pass 
valid_code = np.zeros(8, dtype=int)
print("Zeros should always pass parity check: {}".format(valid_code))
prob = evaluate_potential(ldpc_fg, valid_code)
print("Prob is {}".format(prob))

Zeros should always pass parity check: [0 0 0 0 0 0 0 0]
Prob is 1.0


### Part b: 128-bit LDPC

In [28]:
def get_unary_potential(bit, err):
    phi = np.zeros(shape=(2,))
    phi[bit] = 1 - err 
    phi[1-bit] = err 
    return phi 

def set_unary_factors(fg, msg, err):
    for vnode in fg.get_vnodes():
        idx = int(str(vnode)[1:])
        phi = get_unary_potential(msg[idx-1], err)
        ufnode = nodes.FNode("u" + str(idx))
        ufnode.factor = rv.Discrete(phi, vnode)
        fg.set_node(ufnode)
        fg.set_edge(ufnode, vnode)
    return fg

def transmit(original, err):
    is_flipped = np.random.rand(len(original)) < err
    transmitted = np.where(is_flipped, 1 - original, original)
    return transmitted

In [29]:
original_codeword = np.zeros(256, dtype=int)
transmitted = transmit(original_codeword, err=.1)
transmitted

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
H1 = RegularH(n=256, d_v=4, d_c=8)
print(H1.shape)

(128, 256)


In [31]:
ldpc_fg1 = make_ldpc_fg(H1)
ldpc_fg1 = set_unary_factors(ldpc_fg1, transmitted, 0.1)
beliefs, beliefs_history = get_beliefs(ldpc_fg1, 50)
for (v,b) in beliefs.items():
    print(v, b)

x0 [0.9382336 0.0617664]
x1 [0.98573046 0.01426954]
x2 [0.96560025 0.03439975]
x3 [0.96020859 0.03979141]
x4 [0.94906486 0.05093514]
x5 [0.84875402 0.15124598]
x6 [0.78713697 0.21286303]
x7 [0.54833453 0.45166547]
x8 [0.97351005 0.02648995]
x9 [0.96310876 0.03689124]
x10 [0.92652347 0.07347653]
x11 [0.93929989 0.06070011]
x12 [0.94835145 0.05164855]
x13 [0.96960206 0.03039794]
x14 [0.92530571 0.07469429]
x15 [0.98929771 0.01070229]
x16 [0.83992792 0.16007208]
x17 [0.92035726 0.07964274]
x18 [0.17738013 0.82261987]
x19 [0.88137217 0.11862783]
x20 [0.9528137 0.0471863]
x21 [0.90034823 0.09965177]
x22 [0.98608022 0.01391978]
x23 [0.04109133 0.95890867]
x24 [0.97208951 0.02791049]
x25 [0.97433546 0.02566454]
x26 [0.91045288 0.08954712]
x27 [0.9308438 0.0691562]
x28 [0.84146594 0.15853406]
x29 [0.96842479 0.03157521]
x30 [0.93399665 0.06600335]
x31 [0.95152804 0.04847196]
x32 [0.98273027 0.01726973]
x33 [0.8352225 0.1647775]
x34 [0.97479882 0.02520118]
x35 [0.95487331 0.04512669]
x36 [0.916

### Part E 

In [ ]:
img = cv2.imread('binary-image.png',2)
img = cv2.resize(img, (40, 40))
ret, bw_img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
#cv2.imshow("Binary Image",bw_img)
plt.imshow(bw_img)

In [ ]:
img_code = (bw_img / 255).astype(int)
img_code_flattened = img_code.flatten()
img_code_transmitted = transmit(img_code_flattened, err=0.05)
transmitted_reshaped = img_code_transmitted.reshape(img_code.shape)
plt.imshow(transmitted_reshaped)

In [ ]:
H2 = RegularH(n=256, d_v=64, d_c=128)
print(H2.shape)

In [ ]:
ldpc_fg2 = make_ldpc_fg(H2)

In [ ]:
H2.mean()

## Q4 - Messsage passing on a tree

In [ ]:
from scipy.stats import norm

In [ ]:
norm.pdf(50, loc=50, scale=np.sqrt(10))

In [ ]:
norm.pdf(50, loc=60, scale=np.sqrt(10))

In [ ]:
norm.pdf(60, loc=50, scale=np.sqrt(10))

In [ ]:
norm.pdf(60, loc=60, scale=np.sqrt(10))